# 神经网络

## 感知机


感知机模型的结构如图所示，它接受长度为𝑛的一维向量$X = [x_1, x_2, … , x_n]$，每个输入节点通过权值为$w_i, i\in[1, n]$的连接汇集为变量$z$，即：
$$z = w_1x_1 + w_2x_2 + ⋯ + 𝑤_nx_n + b$$
其中𝑏称为感知机的偏置(Bias)，一维向量$w = [w_1, w_2, … , w_n]$称为感知机的权值(Weight)，$𝑧$称为感知机的净活性值(Net Activation)。

![](./感知机.png)

向量形式:
$$z = w^Tx + b$$
感知机是线性模型，并不能处理线性不可分问题。通过在线性模型后添加激活函数后得到活性值(Activation)$\alpha$:
$$\alpha = \sigma(z) = \sigma(w^T + b)$$

$\sigma$激活函数可以是阶跃函数(Step function)或符号函数(Sign function)

添加激活函数后，感知机可以用来完成二分类任务。阶跃函数和符号函数在𝑧 = 0处是不连续的，其他位置导数为 0，无法利用梯度下降算法进行参数优化。

## 全连接层

![](./全连接层.png)

整个网络层关系:
$$
\begin{bmatrix} o_1 & o_2\end{bmatrix} = 
\begin{bmatrix} x_1 & x_2 & x_3 \end{bmatrix} @
\begin{bmatrix} w_{11} & w_{12} \\ w_{21} & w_{22} \\ w_{31} & w_{32}\end{bmatrix} +
\begin{bmatrix} b_1 & b_2\end{bmatrix}
$$
即 $$O = X @ W + b$$
输入矩阵X的shape定义为$[b, d_{in}]$，𝑏为样本数量，此处只有1个样本参与前向运算，$d_{in}$为输入节点数；权值矩阵 W 的 shape 定义为$[d_{in}, d_{out}]$，$𝑑_{out}$为输出节点数，偏置向量 b的 shape 定义为$[𝑑_{out}]$。

由于每个输出节点与全部的输入节点相连接，这种网络层称为`全连接层(Fully-connected Layer)`，或者`稠密连接层(Dense Layer)`，𝑾矩阵叫做全连接层的权值矩阵，𝒃向量叫做全连接层的偏置向量。

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# Default parameters for plots
matplotlib.rcParams['font.size'] = 20
matplotlib.rcParams['figure.titlesize'] = 20
matplotlib.rcParams['figure.figsize'] = [9, 7]
matplotlib.rcParams['font.family'] = ['Noto Sans CJK JP']
matplotlib.rcParams['axes.unicode_minus']=False 

gpus = tf.config.experimental.list_physical_devices("GPU")
try:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
except RuntimeError as e:
    print(e)

In [ ]:
# 使用张量方式实现全层连接

x = tf.random.normal([2, 784])
w1 = tf.Variable(tf.random.truncated_normal([784, 256]))
b1 = tf.Variable(tf.zeros([256]))
o1 = x @ w1 + b1
o1 = tf.nn.relu(o1)

In [ ]:
# tensorflow layers层方式实现
from tensorflow.keras import layers

x = tf.random.normal([4, 28 * 28])
# 创建全连接层，指定输出节点数和激活函数
fc = layers.Dense(256, activation=tf.nn.relu)
h1 = fc(x)  # 输入的节点数在fc(x)计算时自动获取
h1

In [ ]:
fc.kernel  # 获取 Dense 类的权值矩阵 w

In [ ]:
fc.bias  # 获取 Dense 类的偏置向量

In [ ]:
fc.trainable_variables  # 返回待优化参数列表
# fc.non_trainable_variables

## 神经网络
通过层层堆叠全连接层，保证前一层的输出节点数与当前层的输入节点数匹配,即可堆叠出任意层数的网络-神经网络

![](./神经网络.png)

In [ ]:
x = tf.random.normal([2, 784])

### 使用张量实现

In [ ]:
w1 = tf.Variable(tf.random.truncated_normal([784, 256]))
b1 = tf.Variable(tf.zeros([256]))
w2 = tf.Variable(tf.random.truncated_normal([256, 128]))
b2 = tf.Variable(tf.zeros([128]))
w3 = tf.Variable(tf.random.truncated_normal([128, 64]))
b3 = tf.Variable(tf.zeros([64]))
w4 = tf.Variable(tf.random.truncated_normal([64, 10]))
b4 = tf.Variable(tf.zeros([10]))

In [ ]:
with tf.GradientTape() as tape:
    h1 = x @ w1 + b1
    h1 = tf.nn.relu(h1)
    h2 = h1 @ w2 + b2
    h2 = tf.nn.relu(h2)
    h3 = h2 @ w3 + b3
    h3 = tf.nn.relu(h3)
    out = h3 @ w4 + b4
    # loss = ...
    # 最后一层是否需要添加激活函数通常视具体的任务而定，这里加不加都可以
# grads = tape.gradient(loss, [])

### 使用层方式实现

In [ ]:
from tensorflow.keras import layers, Sequential
# 隐藏层
fc1 = layers.Dense(256, activation=tf.nn.relu)
fc2 = layers.Dense(128, activation=tf.nn.relu)
fc3 = layers.Dense(64, activation=tf.nn.relu)
# 输入层
fc4 = layers.Dense(10, activation=None)

h1 = fc1(x)
h2 = fc2(h1)
h3 = fc3(h2)
out = fc4(h3)
out

In [ ]:
# 使用Sequential封装
model = Sequential([
    layers.Dense(256, activation=tf.nn.relu), 
    layers.Dense(128, activation=tf.nn.relu), 
    layers.Dense(64, activation=tf.nn.relu), 
    layers.Dense(10, activation=tf.nn.relu), 
])
out = model(x)
out

### 优化目标
我们把神经网络从输入到输出的计算过程叫做`前向传播(Forward Propagation)`或前向计算。神经网络的前向传播过程，也是数据张量(Tensor)从第一层流动(Flow)至输出层的过程，即从输入数据开始，途径每个隐藏层，直至得到输出并计算误差，这也是 TensorFlow框架名字由来。

前向传播最后一步, 误差计算:

$$ L = g(f_{\theta}(x), y)$$

优化目标:
$$\theta^*= \mathop{argmin}_{\theta} g(f_{\theta}(x), y), x\in D^{train}$$
一般采用误差反向传播(Backward Propagation，简称 BP)算法来求解网络参数𝜃的梯度信息，并利用梯度下降(Gradient Descent，简称 GD)算法迭代更新参数:
$$\theta'= \theta - \eta \cdot \nabla_{\theta}L$$

### 各种激活函数


Sigmoid函数

也叫logistic函数:
$$ Sigmoid(x) = \sigma(x)= \frac {1} {1+e^{-x}}$$

In [ ]:
x = tf.linspace(-6., 6, 100)
plt.plot(x, tf.nn.sigmoid(x))
ax = plt.gca()

ax.spines['left'].set_position(('data', 0))
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')

In [ ]:
# 像 Dense 层一样将ReLU 函数作为一个网络层添加到网络中
# layers.ReLU()

ReLU函数(REctified Linear Unit，修正线性单元)

Sigmoid 函数在输入值较大或较小时容易出现梯度值接近于 0 的现象，称为梯度弥散现象。出现`梯度弥散`(梯度消失)现象时，网络参数长时间得不到更新，导致训练不收敛或停滞不动的现象发生，较深层次的网络模型中更容易出现梯度弥散现象.ReLU 对小于 0 的值全部抑制为 0；对于正数则直接输出，这种单边抑制特性来源于生物学.
$$Relu(x) = max(0, x)$$

In [ ]:
x = tf.linspace(-6., 6, 100)
plt.plot(x, tf.nn.relu(x))
ax = plt.gca()
ax.spines['left'].set_position(('data', 0))
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')

LeakyReLU
ReLU 函数在𝑥 < 0时导数值恒为 0，也可能会造成梯度弥散现象，为了克服这个问题，LeakyReLU 函数被提出:
$$
LeakyReLU = \begin{cases} x \quad x \geq 0  \\ px \quad x < 0\end{cases} \\ 
g(z) = max(pz, z)
$$
其中𝑝为用户自行设置的某较小数值的超参数，如 0.02 等。当𝑝 = 0时，LeayReLU 函数退化为 ReLU 函数；当𝑝 ≠ 0时，𝑥< 0处能够获得较小的导数值𝑝，从而避免出现梯度弥散现象.


In [ ]:
x = tf.linspace(-6., 6, 100)
plt.plot(x, tf.nn.leaky_relu(x, alpha=0.1))
ax = plt.gca()
ax.spines['left'].set_position(('data', 0))
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')

Tanh函数能将数据压缩到(-1, 1)区间
$$
tanh(x) = \frac {e^x - e^{-x}} {e^x + e^{-x}} = 2 \cdot sigmoid(2x) - 1
$$

In [ ]:
x = tf.linspace(-6., 6, 100)
# tf.nn.tanh
plt.plot(x, tf.tanh(x))
ax = plt.gca()
ax.spines['left'].set_position(('data', 0))
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')

SoftPlus函数
$$
f(x) =\zeta(x) = ln(1+e^x)
$$
它是ReLU函数的平滑版本, 值域为$(0, +\infty)$
一些其他性质:
$$
log\sigma(x) = -\zeta(-x) \\
\frac {d} {dx}\zeta(x) = \sigma(x) \\
\zeta(x) = \int_{-\infty}^x \sigma(y)dy  \\
\zeta(x) - \zeta(-x) = x
$$

In [ ]:
x = tf.linspace(-10., 10., 100)
plt.plot(x, tf.nn.softplus(x))
# ax = plt.gca()


## 输出层的设计
最后一层需要根据具体的任务场景来决定是否使用激活函数, 以及使用什么类型的激活函数等:

1. 普通实数空间$o_i \in R^d$, 这一类问题比较普遍，像正弦函数曲线预测、年龄的预测、股票走势的预测等都属于整个或者部分连续的实数空间，输出层可以不加激活函数
1. $o_i \in [0, 1]$, 输出值特别地落在\[0, 1\]的区间，如图片生成，图片像素值一般用\[0, 1\]区间的值表示；或者二分类问题的概率，如硬币正反面的概率预测问题, 输出层可以只设一个节点, 表示事件发生的概率.可以使用Sigmoid函数
1. $o_i \in [0, 1]$, 且$\sum_i o_i = 1$, 常见的如多分类问题，如 MNIST 手写数字图片识别，图片属于 10 个类别的概率之和应为 1。在输出层添加Softmax函数实现
1. $o_i \in [-1, 1]$, 可以简单的使用tf.tanh函数

In [ ]:
# softmax 函数
z = tf.constant([2, 1, 0.1])
tf.nn.softmax(z)
# 添加 Softmax 层
# layers.Softmax(axis=-1)

在 Softmax 函数的数值计算过程中，容易因输入值偏大发生数值溢出现象；在计算交
叉熵时，也会出现数值溢出的问题。为了数值计算的稳定性，TensorFlow 中提供了一个统
一的接口，将 Softmax 与交叉熵损失函数同时实现，同时也处理了数值不稳定的异常，一
般推荐使用这些接口函数，避免分开使用 Softmax 函数与交叉熵损失函数。函数式接口为
`tf.keras.losses.categorical_crossentropy(y_true, y_pred, from_logits=False)`，其中 y_true 代表了
One-hot 编码后的真实标签，y_pred 表示网络的预测值，当 from_logits 设置为 True 时，
y_pred 表示须为未经过 Softmax 函数的变量 z；当 from_logits 设置为 False 时，y_pred 表示
为经过 Softmax 函数的输出。为了数值计算稳定性，一般设置 from_logits 为 True，此时
tf.keras.losses.categorical_crossentropy 将在内部进行 Softmax 函数计算，所以不需要在模型
中显式调用 Softmax 函数

In [ ]:
z = tf.random.normal([2, 10])  # 模拟输出层
y_onehot = tf.one_hot(tf.constant([1, 3]), depth=10)
# 未调用softmax函数时, 设置from_logists=True
loss = tf.keras.losses.categorical_crossentropy(y_onehot, z, from_logits=True)
loss = tf.reduce_mean(loss)  # 计算平均交叉熵损失
loss

In [ ]:
criteon = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
loss = criteon(y_onehot, z)
loss

## 误差计算

常见的误差函数有均方差、交叉熵、KL 散度、Hinge Loss 函数等，其中均方差函数和交叉熵函数在深度学习中比较常见，均方差函数主要用于回归问题，交叉熵函数主要用于分类问题。

### 均方差(Mean Squared Error, MSE)

$$
MSE(y, o) = \frac 1 {d_{out}} \sum^{d_{out}}_{i=1}(y_i - o_i)^2
$$

均方差误差函数广泛应用在回归问题中，实际上，分类问题中也可以应用均方差误差函数。在 TensorFlow 中，可以通过函数方式或层方式实现 MSE 误差计算

In [ ]:
from tensorflow import keras

out = tf.random.normal([2, 10])
y_onehot = tf.one_hot(tf.constant([1, 3]), depth=10)
loss = keras.losses.mse(y_onehot, out)  # 每个样本的MSE
loss

In [ ]:
tf.reduce_mean(loss)  # 计算 batch 均方差

In [ ]:
# 以layer的方式实现
criterion = keras.losses.MeanSquaredError()
loss = criterion(y_onehot, out)
loss

### 交叉熵误差

某个分布P(i)的熵(Entropy)定义为:
$$H(P) = -\sum_iP(i)log_2P(i)$$

In [ ]:
# 4类分类问题, 四种等可能情况时 熵
p = tf.constant([0.25, 0.25, 0.25, 0.25])
entropy = tf.reduce_sum(-tf.math.log(p) / tf.math.log([2., 2, 2, 2]) * p )
entropy

交叉熵(Cross Entropy):
$$H(p||q) = -\sum_i p(i)log_2q(i)$$
通过变换，交叉熵可以分解为𝑝的熵𝐻(𝑝)和𝑝与𝑞的 KL 散度(Kullback-Leibler Divergence)的和
$$H(p||q) = H(p) + D_{KL}(p||q)$$
KL散度:
$$D_{KL}(p||q) = \sum_i p(i)log(\frac {p(i)}{q(i)})$$
是用于衡量 2 个分布之间距离的指标.当p=q时,$D_{KL}(p||q)$取得最小值, p和q之间的差距越大, $D_{KL}(p||q)$也就越大.

需要注意的是，交叉熵和 KL 散度都不是对称的.  
交叉熵可以很好地衡量 2 个分布之间的“距离”。特别地，当分类问题中y的编码分布𝑝采用One-hot编码𝒚时：𝐻(𝑝) = 0, 此时
$$H(p||q) = H(p) + D_{KL}(p||q)= D_{KL}(p||q)$$
推导分类问题中的交叉熵的计算表达式:
$$ H(p||q) = D_{KL}(p||q) = \sum_j y_j log(\frac {y_j} {o_j}) \\
= 1 \cdot log\frac {1}{o_i} + \sum_{i \neq j} 0 \cdot log\frac {0}{o_j} \\
= -logo_i
$$
其中𝑖为 One-hot 编码中为 1 的索引号，也是当前输入的真实类别。可以看到，ℒ只与真实类别𝑖上的概率$𝑜_𝑖$有关，对应概率$𝑜_𝑖$越大，𝐻(𝑝||𝑞)越小。当对应类别上的概率为 1 时，交叉熵𝐻(𝑝||𝑞)取得最小值 0，此时网络输出𝒐与真实标签𝒚完全一致，神经网络取得最优状态.
最小化交叉熵损失函数的过程也是最大化正确类别的预测概率的过程.

In [ ]:
z = tf.random.normal([10])  # 模拟输出层
z

In [ ]:
s1 = tf.nn.softmax(z)
s1

In [ ]:
-tf.math.log(s1[1])

In [ ]:
y_onehot = tf.one_hot(tf.constant([1]), depth=10)
# 计算交叉熵
# 未调用softmax函数时, 设置from_logists=True
loss = tf.keras.losses.categorical_crossentropy(y_onehot, z, from_logits=True)
loss

## 神经网络类型

- 卷积神经网络(Convolutional Neural Network, CNN): 应用于计算机视觉, 如图片分类;
- 循环神经网络(Recurrent Neural Network, RNN): 序列信号处理, 如理解文本数据, NLP;
- 注意力(机制)网络(Attention Mechanism): 自然语言处理;
- 图卷积神经网络(Graph Convolution Network, GCN): 处理社交网络、通信网络、蛋白质分子结构等一系列的不规则空间拓扑结构的数据

## 汽车耗油项目实战

In [ ]:
import seaborn as sns 
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses, Sequential

In [ ]:
# 在线下载汽车效能数据集
dataset_path = keras.utils.get_file("auto-mpg.data", 
"http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")

In [ ]:
# 利用 pandas 读取数据集，字段有效能（公里数每加仑），气缸数，排量，马力，重量, 加速度，型号年份，产地
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight', 
    'Acceleration', 'Model Year', 'Origin']

In [ ]:
raw_daraset = pd.read_csv(dataset_path, names=column_names, 
    na_values='?', comment='\t', sep=' ', skipinitialspace=True)

In [ ]:
dataset = raw_daraset.copy()

In [ ]:
# 查看部分数据
dataset.head()

In [ ]:
# 查看 NaN
dataset.isnull().sum()

In [ ]:
# 直接舍弃缺失的数据
dataset = dataset.dropna()
dataset.isna().sum()

In [ ]:
# 类别类型数据 # 处理类别型数据，其中origin列代表了类别1,2,3,分布代表产地：美国、欧洲、日本
# 其弹出这一列
dataset['Origin']

In [ ]:
origin = dataset.pop('Origin')
dataset['USA'] = (origin == 1) * 1.0
dataset['Europe'] = (origin == 2) * 1.0
dataset['Japan'] = (origin == 3) * 1.0
dataset.tail()

In [ ]:
columns = dataset.columns
columns

In [ ]:
# 划分数据集 训练:测试 = 8:2
X_train = dataset.sample(frac=0.8, random_state=0)
X_test = dataset.drop(X_train.index)

In [ ]:
sns.pairplot(X_train[["Cylinders", "Displacement", "Weight", "MPG"]], 
diag_kind="kde")


In [ ]:
# MPG 数据作为 labels 
y_train = X_train.pop('MPG')
y_test = X_test.pop('MPG')
y_train

In [ ]:
# 转成numpy
X_train, X_test = X_train.values, X_test.values
y_train, y_test = y_train.values, y_test.values
y_train

In [ ]:
X_train_normal = (X_train - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)
X_train_normal

In [ ]:
X_test_normal = (X_test - np.mean(X_test, axis=0)) / np.std(X_test, axis=0)
X_test_normal

In [ ]:
print(X_train_normal.shape, y_train.shape)
print(X_test_normal.shape, y_test.shape)

In [ ]:
# 构建Dataset对象
train_db = tf.data.Dataset.from_tensor_slices((X_train_normal, y_train))
train_db = train_db.shuffle(100).batch(32)

In [ ]:
# 创建一个3层的回归网络 
# 输入𝑿的特征共有 9 种，因此第一层的输入节点数为 9。第一层、第二层的
# 输出节点数设计为64和64，由于只有一种预测值，输出层输出节点设计为 1

class Network(keras.Model):
    def __init__(self):
        super().__init__()
        self.model = Sequential([
            layers.Dense(64, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(1)
        ])
        # self.fc1 = layers.Dense(64, activation='relu')
        # self.fc2 = layers.Dense(64, activation='relu')
        # self.fc3 = layers.Dense(1)
    
    # 在前向计算函数 call 中实现自定义网络类的计算逻辑即可
    def call(self, inputs, training=None, mask=None):
        out = self.model(inputs)
        return out

In [ ]:
model = Network()  # 
# 通过 build 函数完成内部张量的创建，其中 4 为任意设置的 batch 数量，9 为输入特征长度
model.build(input_shape=(4, 9))
model.summary() # 打印网络信息

In [ ]:
9 * 64 + 64 + 64 * 64 + 64 + 64 * 1 + 1

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(0.001)  # 创建优化器, 指定学习率

$$mae = \frac 1{d_{out}} \sum_i |y_i - o_i|$$

In [ ]:
train_mae_losses = []
train_losses = []
test_mae_losses = []
for epoch in range(200):
    for step, (x, y) in enumerate(train_db):
        with tf.GradientTape() as tape:
            out = model(x)
            loss = tf.reduce_mean(losses.MSE(y, out))  # 最小化的目标MSE 
            mae_loss = tf.reduce_mean(losses.MAE(y, out))  # MAE用平均绝对误差
        if step % 10 == 0:
            print(epoch, step, float(loss))
        
        # 自动计算梯度
        train_losses.append(loss)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    # 记录每个epoch 的mae误差
    train_mae_losses.append(float(mae_loss))
    out = model(X_test_normal)
    test_mae_losses.append(tf.reduce_mean(losses.MAE(out, y_test)))

In [ ]:
plt.plot(train_mae_losses, label='Train')
plt.plot(test_mae_losses, label='Test')
# plt.ylim(1, 10)
plt.legend()
plt.show()

In [ ]:
plt.plot(train_losses)